# Character-level RNN model 

Author: Ted Moskovitz, adapted from framework by Alex Beatson (site: http://www.cs.princeton.edu/~abeatson/) 

Data I/O adapted from Andrej Karpathy's CharRNN gist: https://gist.github.com/karpathy/d4dee566867f8291f086

# Description and Purpose

Perhaps the primary obstacle standing in the way of undeciphered writing systems is the limited size of their extant corpora. The goal of this project is to learn the inter-character relationships in an unknown script, and use that understanding (encoded in the parameters of a machine learning model) to produce more text in that writing system. The hope is that this artificially generated text can sufficiently capture the dynamics of the unknown writing system such that it can be used by linguists to increase their understanding of the script itself. 

Deep learning approaches like this one require a large volume of data on which to train. This poses a problem, as the limited size of extant corpora is the very issue that I am trying to solve. Fortunately, geographical and historical knowledge of undeciphered scripts often provides clues as to which other (hopefully deciphered) writing systems they are related. Related languages (and writing systems) often have similar character dynamics, and I take advantage of that fact. For the first 80% of training time, I train the network on a closely related language/writing system, and then switch to the unknown writing system for the final 20% of training time, sampling model output periodically along the way. 

As appropriately digitized corpora are hard to find for ancient scripts, below I display a proof-of-concept that this appraoch could work. For the first 80% of training time, the network is trained on Harry Potter, and for the final 20% I switch to a roughly 8,000 character corpus of disjoint Shakespeare (a different sort of document, but also quite related). The corpus size (~8,000 characters) was chosen because that is roughly the size of the surviving corpus of the  undeciphered Cretan script known as Linear A. The Shakespeare training corpus is cut into disjoint fragments of on average ~40 characters to reflect the fragmented nature of the surviving samples of Linear A, which is broken up onto often unrelated clay tablets. 

## Design notes:

The class RNN wraps a TF model (including hyperparameters, Variables and the computation graph).

Non-TF computation (except feeding inputs) happens outside the class.

Class methods preceeded by underscore (e.g. _init_params, _rnn_step) contain TF functions and are used to build the computation graphs for training and sampling. Placeholders are defined in `_build_graph`. These 'private' methods should be called within RNN.

Methods without underscore (`run_train`, `run_sample`) run a TF session and feed placeholder values but otherwise contain no TF functions. These 'public' methods should be called outside RNN.

In [21]:
import numpy as np
import tensorflow as tf
import random

In [2]:
class RNN(object):

    def __init__(self, batch_size, embedding_size, hidden_size, vocab_size, seq_length,
                 learning_rate, decay_steps, decay_factor, sample_len):
        ''' Set the hyperparameters and define the computation graph.
        '''

        ''' hyperparameters '''

        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size # number of chars in vocab
        self.seq_length = seq_length # number of steps to unroll the RNN for
        self.initial_learning_rate = learning_rate
        self.decay_steps = decay_steps
        self.decay_factor = decay_factor
        self.sample_len = sample_len

        # this var keeps track of the train steps within the RNN
        self.global_step = tf.Variable(0, trainable=False)

        ''' create vars and graph '''

        self._init_params()

        self._build_graph()


    def _init_params(self):
        '''Create the model parameters'''
        
        # We learn an embedding for each character jointly with the other model params
        self.embedding = tf.Variable(tf.random_normal([self.vocab_size, self.embedding_size],
                                                      mean=0, stddev=0.2))

        self.Uf = tf.Variable(tf.random_normal([self.embedding_size, self.hidden_size],
                                       mean=0, stddev=0.2))
            
        self.Wf = tf.Variable(tf.random_normal([self.hidden_size, self.hidden_size],
                                               mean=0, stddev=0.2))
        
        self.bf = tf.Variable(tf.zeros([1, self.hidden_size]))
        
        self.Ui = tf.Variable(tf.random_normal([self.embedding_size, self.hidden_size],
                                       mean=0, stddev=0.2))
            
        self.Wi = tf.Variable(tf.random_normal([self.hidden_size, self.hidden_size],
                                               mean=0, stddev=0.2))
        
        self.bi = tf.Variable(tf.zeros([1, self.hidden_size]))
        
        self.Uo = tf.Variable(tf.random_normal([self.embedding_size, self.hidden_size],
                                       mean=0, stddev=0.2))
            
        self.Wo = tf.Variable(tf.random_normal([self.hidden_size, self.hidden_size],
                                               mean=0, stddev=0.2))
        
        self.bo = tf.Variable(tf.zeros([1, self.hidden_size]))
        
        self.Uc = tf.Variable(tf.random_normal([self.embedding_size, self.hidden_size],
                                       mean=0, stddev=0.2))
            
        self.Wc = tf.Variable(tf.random_normal([self.hidden_size, self.hidden_size],
                                               mean=0, stddev=0.2))
        
        self.bc = tf.Variable(tf.zeros([1, self.hidden_size]))


        self.V = tf.Variable(tf.random_normal([self.hidden_size, self.vocab_size],
                                               mean=0, stddev=0.2))
        
        self.by = tf.Variable(tf.zeros([1, self.vocab_size]))


    def _rnn_step(self, x, h, c):
        '''Performs RNN computation for one timestep:
        takes a previous x and h, and computes the next x and h.
        
        In practical applications, you should almost always use TensorFlow's built-in RNN cells,
        from tf.contrib.rnn. However for teaching purposes we are writing the RNN from scratch here.
        '''
        f = tf.nn.sigmoid(tf.matmul(x, self.Uf) + tf.matmul(h, self.Wf) + self.bf)
        i = tf.nn.sigmoid(tf.matmul(x, self.Ui) + tf.matmul(h, self.Wi) + self.bi)
        o = tf.nn.sigmoid(tf.matmul(x, self.Uo) + tf.matmul(h, self.Wo) + self.bo)
        c = f * c + i * tf.tanh(tf.matmul(x, self.Uc) + tf.matmul(h, self.Wc) + self.bc)
        h = o * tf.tanh(c)
        y = tf.matmul(h, self.V) + self.by

        return y, h, c

    
    def _forward(self, inputs):
        '''Performs the forward pass for all timesteps in a sequence.
        '''
        # Create list to hold y
        y = [_ for _ in range(self.seq_length)]

        # Create zero-d initial hidden state
        h = tf.zeros([self.batch_size, self.hidden_size])
        c = tf.zeros([self.batch_size, self.hidden_size])

        for t in range(self.seq_length):
            x = tf.nn.embedding_lookup(self.embedding, inputs[:, t])
            y[t], h, c = self._rnn_step(x, h, c)

        return y

    
    def _sample_one(self, input_character, input_hidden, input_cell, temperature):
        '''Sample the single next character in a sequence.

        We can use this to sample sequences of any length w/o having to alter
        the tensorflow graph.'''

        # We expand dims because tf expects a batch
        character = tf.expand_dims(input_character, 0)

        # Get the embedding for the input character
        x = tf.nn.embedding_lookup(self.embedding, character)

        # Take a single rnn step
        y, h, c = self._rnn_step(x, input_hidden, input_cell)

        # Dividing the unnormalized probabilities by the temperature before 
        # tf.multinomial is equivalent to adding temperature to a softmax
        # before sampling
        y_temperature = y / temperature

        # We use tf.squeeze to remove the unnecessary [batch, num_samples] dims
        # We do not manually softmax - tf.multinomial softmaxes the tensor we pass it
        next_sample = tf.squeeze(tf.multinomial(y_temperature, 1))

        return next_sample, h, c, y


    def _build_graph(self):
        '''Build the computation graphs for training and sampling.

        All placeholders to be fed in and ops / tensors to be run / output defined here.'''


        '''Sampling and test graph'''
        self.sample_input_char = tf.placeholder(dtype=tf.int32, shape=[])
        self.sample_input_hidden = tf.placeholder(dtype=tf.float32, shape=[1, self.hidden_size])
        self.sample_input_cell = tf.placeholder(dtype=tf.float32, shape=[1, self.hidden_size])

        self.test_char = tf.placeholder(dtype=tf.int32, shape=[])
        
        self.temperature = tf.placeholder_with_default(1.0, [])

        self.next_sample, self.next_hidden, self.next_cell, self.next_predictions = self._sample_one(
            self.sample_input_char, self.sample_input_hidden, self.sample_input_cell, self.temperature)
        
        self.next_softmax_predictions = tf.nn.softmax(self.next_predictions)
                
        self.test_char_prob = tf.reduce_sum(self.next_softmax_predictions * tf.one_hot(
            tf.expand_dims(self.test_char, axis=0), depth=self.vocab_size))
        
        # Get cross entropy in base 2
        # log_2 (x) =  log_e (x) / log_e(2)
        self.binary_xentropy = - tf.log(self.test_char_prob) / tf.log(2.0)


        '''Training graph'''
        self.inputs = tf.placeholder(dtype=tf.int32, shape=[None, self.seq_length])
        self.targets = tf.placeholder(dtype=tf.int32, shape=[None, self.seq_length])
        self.predictions = self._forward(self.inputs)

        cost_per_timestep_per_example = [
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=self.predictions[t],
                    labels=self.targets[:, t])
                for t in range(self.seq_length)
        ]

        # Use reduce_mean rather than reduce_sum over the examples in batch so that
        # we don't need to change the learning rate when we change the batch size.
        cost_per_timestep = [tf.reduce_mean(cost) for cost in cost_per_timestep_per_example]

        # Use reduce_mean here too so we don't need to change the learning rate when
        # we change number of timesteps.
        self.cost = tf.reduce_mean(cost_per_timestep)

        # Decay the learning rate according to a schedule.
        self.learning_rate = tf.train.exponential_decay(self.initial_learning_rate,
                                                        self.global_step,
                                                        self.decay_steps,
                                                        self.decay_factor)
        
        self.train_step = tf.train.RMSPropOptimizer(self.learning_rate).minimize(
            self.cost, global_step=self.global_step)


        '''Finished creating graph: start session and init vars'''
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())


    def run_train(self, input_chars, target_chars):
        '''Call this from outside the class to run a train step'''
        cost, lr, _ = self.sess.run([self.cost, self.learning_rate, self.train_step],
                                   feed_dict={
                                       self.inputs: input_chars,
                                       self.targets: target_chars
                                   })
        return cost, lr


    def run_sample(self, n, starter_character, temperature=1.0):
        '''Call this from outside the class to sample a length-n sequence from the model'''

        sampled_chars = [_ for _ in range(n)]
        current_char = starter_character
        h = np.zeros([1, self.hidden_size])
        c = np.zeros([1, self.hidden_size])

        for i in range(n):

            current_char, h, c = self.sess.run(
                [self.next_sample, self.next_hidden, self.next_cell],
                feed_dict={
                    self.sample_input_char: current_char,
                    self.sample_input_hidden: h,
                    self.sample_input_cell: c, 
                    self.temperature: temperature})

            sampled_chars[i] = current_char

        return sampled_chars
    
    def run_test(self, test_chars, primer_seq=None):
        '''Call this from outside the class to find the cross entropy on a dataset.
        test_chars and primer_seq should be lists of ints.
        If primer_seq is not None, "prime" the RNN by feeding primer_seq through it
        before beginning testing. (primer_seq should be the characters
        which appear immediately before test_chars).
        We dont report xentropy for the first char of test chars - this shouldnt
        matter for any reasonable test set.'''
        
        xentropy_accum = 0.0
        h = np.zeros([1, self.hidden_size])
        cs = np.zeros([1, self.hidden_size])
        
        if primer_seq is not None:
            for c in primer_seq:
                h, cs = self.sess.run(
                    [self.next_hidden, self.next_cell],
                    feed_dict={
                        self.sample_input_char: c,
                        self.sample_input_hidden: h,
                        self.sample_input_cell: cs
                    })
        
        for i in range(len(test_chars) - 1):
            xentropy, h, cs = self.sess.run(
                [self.binary_xentropy, self.next_hidden, self.next_cell],
                feed_dict={
                    self.sample_input_char: test_chars[i],
                    self.sample_input_hidden: h,
                    self.sample_input_cell: cs,
                    self.test_char: test_chars[i+1]
                })

            xentropy_accum += (xentropy / len(test_chars))
        
        xentropy_avg = xentropy_accum 
        
        return xentropy_avg 

In [3]:
hp = open('hp1.txt', 'r').read()
hp = hp[492:] # remove bs at front of file
hp = ''.join([i for i in hp if not i.isdigit()])
hp = ''.join([i for i in hp if not i=='\\'])
print hp[-100:]

re not allowed to use magic at home. I'm 
going to have a lot of fun with Dudley this summer...' 

}


In [1]:
# partitions a given list into n fragments of roughly equal size
def partition(lst, n): 
    division = len(lst) / float(n) 
    return [ lst[int(round(division * i)): int(round(division * (i + 1)))] for i in xrange(n) ]

In [2]:
# generate random fragments
def gen_fragments(s, n=200):
    s = s.split(' ')
    p = partition(s, n)
    for l in xrange(len(p)):
        p[l] = ' '.join(p[l])
    random.shuffle(p)
    print len(p[1])
    return ' '.join(p)

In [41]:
'''Train and sample from our model'''

# data I/O
data = open('shakespeare.txt', 'r').read() # should be simple plain text file
train_data = data[:int(.007 * len(data))]
train_data = gen_fragments(train_data)

test_data = data[int(.9 * len(data)):]

chars_s = list(set(train_data))
data_size, vocab_size = len(train_data), len(chars_s)

chars_h = list(set(hp))
size_h, vocab_size_h = len(hp), len(chars_h)

print 'Shakespeare has %d characters, %d unique.' % (data_size, vocab_size)
print 'Harry Potter has %d characters, %d unique.' % (size_h, vocab_size_h)
char_to_ix = { ch:i for i,ch in enumerate(chars_s + chars_h) }
ix_to_char = { i:ch for i,ch in enumerate(chars_s + chars_h) }


55
Shakespeare has 7807 characters, 56 unique.
Harry Potter has 473331 characters, 67 unique.


In [42]:
# hyperparameters
embedding_size = 32 # size of embedding
hidden_size = 256 # size of hidden layers of neurons
seq_length = 50 # number of steps to unroll the RNN for
learning_rate = 1e-2
decay_steps = 500
decay_factor = 0.9
sample_len = 500

batch_size = 128

n_train_steps = 10000

# model parameters
rnn = RNN(batch_size, embedding_size, hidden_size, vocab_size+vocab_size_h, 
          seq_length, learning_rate, decay_steps, decay_factor, 
          sample_len)

smooth_loss = -np.log(1.0/(vocab_size+vocab_size_h))*seq_length # loss at iteration 0
for n in range(n_train_steps):
    
    # prepare inputs 
    inputs = np.empty([batch_size, seq_length])
    targets = np.empty([batch_size, seq_length])
    
    if n < .8 * n_train_steps:
        for i in range(batch_size):
            # randomly index into the data for each example in batch
            random_index = int(np.random.rand() * (size_h - seq_length - 1))
            inputs[i, :] = [char_to_ix[ch] for ch in hp[random_index:random_index+seq_length]]
            targets[i, :] = [char_to_ix[ch] for ch in hp[random_index+1:random_index+seq_length+1]]
    else: 
        for i in range(batch_size):
            # randomly index into the data for each example in batch
            random_index = int(np.random.rand() * (data_size- seq_length - 1))
            inputs[i, :] = [char_to_ix[ch] for ch in train_data[random_index:random_index+seq_length]]
            targets[i, :] = [char_to_ix[ch] for ch in train_data[random_index+1:random_index+seq_length+1]]
        
    loss, lr = rnn.run_train(inputs, targets)
    
    # print progress
    if n % 100 == 0: 
        print 'iter %d, loss: %f, learning rate: %f' % (n, loss, lr) 

    # sample from the model now and then
    if n % 1000 == 0:
        sample_ix = rnn.run_sample(sample_len, inputs[0, 0], 1.0)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print '----\n %s \n----' % (txt, )

iter 0, loss: 5.055069, learning rate: 0.010000
----
 |Yr!liCwOKV|T(C PS -O,GiJLIB|tOKyje
Im?sbSiH)I N f:x}bcdFpgpKBf/
mdTmmjU/d.'!-.".T.FlO ta
?DgfwAjjHTntNNzWEAIOn-(RUpEC)ySHiaHJtmDumhtcn"TfcTnJTnU;W-NvIdbN!z!g?cLBmNh. jZukSJ?b}s/F.)}NigcUcOVY.,fjJcRboYCVnDieCloCVqOzqHLCVFonfEEmO.xcliflFhzufxFRzRr?md?c
voj':-vdcpYzpu'T/l}rCtcWqRs:udVCAOOarfygOgYWCMkURo?-,a?fBHHyL::,cuysQU:,geYrl!NtzFivuJ-,EV y!HyC,so!iPRih:
OsO)uNyGegijysFNjMibuyyyWwbsbHCF.BLrmGVHm e)SkNU ewPjpyqCO.xOIJwwISArQN?Ir( ?ep'.y|ep-?vbhjgQ?!JuSRnHHEkShmummm!KfePtE

wNaHakTjx,pVEVg'Ismd? 
----
iter 100, loss: 3.375434, learning rate: 0.009791
iter 200, loss: 2.013649, learning rate: 0.009587
iter 300, loss: 1.473404, learning rate: 0.009387
iter 400, loss: 1.327827, learning rate: 0.009192
iter 500, loss: 1.251814, learning rate: 0.009000
iter 600, loss: 1.258727, learning rate: 0.008812
iter 700, loss: 1.176149, learning rate: 0.008629
iter 800, loss: 1.139212, learning rate: 0.008449
iter 900, loss: 1.155306, learning rate

----
 hy middle Aunt Petunia ansalled boy 
were prouses,' said Hagrid, who was poust in your 
own life was back. He turned his threately is more nasty that he'd have 
such a desk. That's a window to his suddenly at Wood, you see it himself. 

'I'm warning you ' yer hut as if Dumbledore will exactly them toward him. 

But the forest hides, went jollows every fear whooped him water under 
out a work!' said Harry. 'If they would see 
Potter?' Quirrell look stood all he was quill, would win 
the cloak tra 
----
iter 8100, loss: 0.541937, learning rate: 0.001814
iter 8200, loss: 0.300221, learning rate: 0.001777
iter 8300, loss: 0.220901, learning rate: 0.001740
iter 8400, loss: 0.195978, learning rate: 0.001703
iter 8500, loss: 0.185517, learning rate: 0.001668
iter 8600, loss: 0.176588, learning rate: 0.001633
iter 8700, loss: 0.163198, learning rate: 0.001599
iter 8800, loss: 0.150759, learning rate: 0.001566
iter 8900, loss: 0.147915, learning rate: 0.001533
iter 9000, loss: 0.140435, l

**Note: The network appears (qualitatively) to adapt very quickly--and effectively--to the change in training data. This is an early sign of promise for the success of this approach.  **

**Test Data:**

In [43]:
test_ints = np.zeros_like(np.asarray(list(test_data)))
prime_ints = np.zeros((seq_length,))
x= np.asarray(test_data)
for i in xrange(len(test_data)):
    test_ints[i] = char_to_ix[test_data[i]]
for i in xrange(seq_length):
    prime_ints[i] = char_to_ix[train_data[-seq_length+i]]
print rnn.run_test(test_ints, primer_seq=prime_ints)

29.0651873401


# TODO: 

Will calculate perplexity of model trained only on full corpus of Shakespeare, and compare to perplexity of model trained as above. The difference between these two values will provide a quantitative clue as to how effectively the network is able to transfer its learning. More varied data will also be used to test the model.